# Hyper-parameter Tuning

**README**: This notebook mainly focus on mathematical concepts of some hyper-parameters to tune during training/testing a nerual network. Again, made based on Coursera class but some math derivations are made on my own. Don't hesitate to reach out if you find anything that is not right. 

Email: yd1@andrew.cmu.edu

**Background**: Since there are many hyper-parameters with different scale to choose for training a nerual network, it is hard to pre-determine some hyper-parameter values. So training a nerual network is a higly iterative process. 

## Outline
1. Train/Dev/Test Clarification
2. Bias Variance Recipe
3. Vanishing/Exploding Gradients
4. Regularization Techniques
  1. L1/L2 Regularization
  2. Drop-Out
5. Batch Training
6. Optimization
  1. Momentum
  2. RMS-Prop
  3. Adam
7. Learning Rate Decay
8. Batch Normalization
9. Summary of Strategies

## 1. Train/Dev/Test Clarification
Dev set: hold-out cross validation set.
Sometimes there is no "test-set" and pepople refers dev set to test set
<img src="images/train-dev-test.png" style="width:800px;height:150px;">
<p>Traditionally</p>
* 70% train, 30% test
* 60% train, 20% dev, 20% test

<p>Now, having big data allows to allocate more data to trian and less to test. (i.e. 1% of 10 million still a lot)</p>
* usually, 98% train, 1% dev, 1% test (subject to change)

## 2. Bias Variance Recipe
In deep-learning, situation such as high bias and high variance can take place. For example under the assumption the data is not linearly seperable, the classifier is underfit (high bias) for some region with linear decision boundary, and suddenly the classifer overfit in some region (high variance). 

* High Bias(Training performance) -> Larger network(increase flexibility) or (Different NN framework) -> Optimizations
<br />
* High Variance -> More data -> Regularization

##  Normalization Input
I think the image below is very self-explanatory for the importance of normalizing input. After normalizing the input, the gradient descent will converge more quickly to the minimum. 


<img src="images/norm.png" style="width:800px;height:400px;">

## 3. Vanishing/ Exploding Gradients
<img src="images/gradientsVE.png" style="width:800px;height:120px;">
<br />

**Problem** Having a very deep nerual networks will essentially cause numerical unstablity. Assume the bias term is 0 and the activation function is linear(for simplify the concept)
<br />
$$ \hat y = W^{[L]}* W^{[L-1]} \dots * W^{[1]} * X$$
<br />
The initialization of gradients if less than 1 will cause the gradients decay expontentially and similar for case where initialization of gradients that greater than 1.
<br />
**(Partial) Solution** Initialize the input to a hidden unit have variance $ Var(W^{[i]}) = \frac{1}{n}$ parameter using "Xavier initialization" (Or He Initialization, etc. depends on the activation function) difference in terms of scaling parameter.
$$\sqrt{\frac{2}{\text{dimension of the previous layer}}}$$

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import scipy.io

In [3]:
def initialize_parameters_he(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims) - 1 # integer representing the number of layers
     
    for l in range(1, L + 1):
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l-1]) * np.sqrt(2/layers_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layers_dims[l], 1))
        
    return parameters

## 4. Regularization 
There are couple main regularization strategies such L1, L2, Drop-out etc. The L1 and L2 regularization technique is similar to linear regression case where adding a penalty term to the loss function. This section will briefly talk about both L2 regularization and focus on conceputal understanding the Drop out.

### 4.1 - L2 Regularization
The standard way to avoid overfitting is called **L2 regularization**. It consists of appropriately modifying your cost function, from:
$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$
To:
$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} \tag{2}$$
By adding the regularization term: (also called "Frobenius Norm")
$$\frac{\lambda}{2m}\sum\limits_{l = 1}^{L} \lVert W^{[L]}\rVert ^2 = \sum\limits_{i}^{l}\sum\limits_{j}^{l-1}W_{ij}^{[l]}$$


### 4.2 - Drop-out
**dropout** is a widely used regularization technique that is specific to deep learning. 
**It randomly shuts down some neurons in each iteration.** 
<br />
The following video very well demonstrated the process of drop-out and is taken from the coursera assignment.
Note when implementing nerual networks with dropout, the 


<center>
<video width="620" height="440" src="images/dropout1_kiank.mp4" type="video/mp4" controls>
</video>
</center>

**keep_prob** The probability to keep the a neruon. (i.e. on average there are 20% being dropped randomly each time)

**forward propogation with dropout**
The goal is to randomly shutdown some nodes. There are typically 4 steps to do so:
1. create a random matrix, named D here,  with the shape of the output of the activation function.
2. Assign elements in D to be 1 with probability(`keep_prob`), and 0 otherwise. The code to implement is following
</br >

```
for i,v in enumerate(x):
    if v < keep_prob:
        x[i] = 1
    else: # v >= keep_prob
        x[i] = 0
```

```
The pythonic way to do this will be:
X = (X < keep_prob).astype(int)
which is same as above if-else statement
```
</br >
3. Shut down some nodes by applying multiplication.
$$A^{[L]} = A^{[L]}*D^{[L]}$$ 
4. Scale the values of neurons that haven't been shut down, because previously scaled down this factor.
$$A^{[L]} = \frac{A^{{L}}}{\text{keep_prob}}$$

</br >

**Backprop with Dropout**
During the backprop, the previsouly shut down nerouns must be shut down as well. This is acheived by using similar technique but the target to the $dA^{[L]}$
1. Make the gradient update to 0 by multiplying the gradients to mask matrix. 
$$ dA^{[L]} = dA^{[L]} * D^{[L]} $$
2. Scale the neurons that haven't been shut down
$$ dA^{[L]} = \frac{dA^{[L]}}{\text{keep_prob}}$$

In [ ]:
#Example of implementation
D1 = np.random.rand(A1.shape[0], A1.shape[1])           # Step 1: initialize matrix D1 = np.random.rand(..., ...)
D1 = (D1 < keep_prob).astype('int')                     # Step 2: convert entries of D1 to 0 or 1 (using keep_prob as the threshold)
A1 = np.multiply(A1, D1)                                # Step 3: shut down some neurons of A1
A1 = A1/keep_prob                                       # Step 4: scale the value of neurons that haven't been shut down

## 5. Batch Training
One Epoch represents one iteration through the entire data sets. The core idea about batch training is that the gradient descent over entire training examples is too slow because it makes small changes to the coefficients by going through the entire data set. It is useful to divide the data into small batches.
</br>
The two extreme cases are batch size = 1(stochastic gradient descent) and batch size = m (gradient descent), but the optimal solution is to use batch size somewhere ein between. (mini batch)
<img src="images/sgd.png" style="width:750px;height:250px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **SGD vs GD**<br> "+" denotes a minimum of the cost. SGD leads to many oscillations to reach convergence. But each step is a lot faster to compute for SGD than for GD, as it uses only one training example (vs. the whole batch for GD). </center></caption>
<img src="images/minibatch.png" style="width:750px;height:250px;">
<caption><center> <u> <font color='purple'> **Figure 2** </u>: <font color='purple'>  **SGD vs Mini-Batch GD**<br> "+" denotes a minimum of the cost. Using mini-batches in your optimization algorithm often leads to faster optimization. </center></caption>

#### Mini-Batch Training
1. <b>Shuffle</b>: Create a shuffled version of the training set (X, Y)
2. <b>Partition</b>: Partition the shuffled (X, Y) into mini-batches of size mini_batch_size. Note that the number of training examples is not always divisible by mini_batch_size. The last mini batch might be smaller. When the final mini-batch is smaller than the full mini_batch_size. $m-mini_\_batch_\_size \times \lfloor \frac{m}{mini\_batch\_size}\rfloor$

**Note**: The number of data in a single mini-batch generally is power of two (i.e. 2, 4, 8, 16, 32, 64 etc.)

## 6. Optimization
Unlike training a netual network using gradient descent, which the cost function will decrease strict monotonically (if not, there might be a bug), training using mini-batch gradident descent will cause variantion in terms of cost function because its only seeing a subset of data each time (not generalize very well). Thus the cost function will oscillate as shown in the above in figure 2, and sometimes even the cost function will diverge. 
<p>The essential idea to solve this problem is to implement some methods to reduce the vertical variation of the cost function and increase the horizantal steps. There are several optimization methods that can help to reduce this problem.</p>

The core idea is:
```
On each iteration:
    Compute dw, db
    Update dw, db using new optimized gradients
```

### 6.1 Momentum 
<strong> Momentum update </strong> is an approch which can always optimize the convergence rate on deep networks. The idea of momentum is to takes account of past gradients and smooth the cost function, more specifically, smooth the vertical direction and increase the horizantal learning.

$$ V_{dw} = \beta_{1} V_{dw} + (1 - \beta_{1})dw $$
<br>
$$ V_{db} = \beta_{1} V_{db} + (1 - \beta_{1})db $$
<br>
$$ W := W - \alpha V_{dw} $$
<br>
$$ b := b - \alpha b_{db} $$
<br>

**Explain**: The update rule is essentially applying expontentially weighted averge to the coefficients. Since the in the vertical direction, the gradients have different directions, they tends to average to 0 (damped out the oscillation), meanwhile, the horizantal update will still big. 

**Analogy**: There is one analogy where compare the cost minimizing to accerlation of a ball rolling downhill, where the $\beta$ serves as friction and $v_{dw}$ serves as velocity, at same time, $dw$ is the acceleration. So the latter term will cause the ball to build up speed according to the direction of the gradient/slope.

**Note**: 
1. Practically the $\beta$ value equals to 0.9 is pretty robust.
2. The bias correction in expontentially weighted average is usually ignored. 
3. In some iterature, sometimes $(1-\beta)$ is ignored. (dW is scaled up)

### 6.2 RMS-Prop
<strong> RMS-Prop </strong> is yet similar approch to reduce the cost oscillation by restricting in terms of vertical direction. 

$$S_{dw} = \beta_{2}S_{dw} + (1 - \beta_{2}) dw^{2} $$
<br>
$$S_{db} = \beta_{2}S_{db} + (1 - \beta_{2}) db^{2} $$
<br>
$$ W := W - \alpha * \frac{dw}{\sqrt{S_{dw}} + \epsilon}$$
<br>
$$ b := b - \alpha * \frac{db}{\sqrt{S_{db}} + \epsilon}$$
<br>

**Note** The $\epsilon$ here is to ensure the numerical stablity

### 6.3 Adam
Adam, <strong>Adaptive Moment Estimation </strong>, is essentially the combination of momentum and RMS-Prop.

$$v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } $$
<br>
$$v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} $$
<br>
$$s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 $$
<br>
$$s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t}$$
<br>
$$W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}$$
<br>
$$b^{[l]} = b^{[l]} - \alpha \frac{v^{corrected}_{db^{[l]}}}{\sqrt{s^{corrected}_{db^{[l]}}} + \varepsilon}$$

**Note**:
1. Take bias correction term into consideration.
2. Some of common choice for hyper parameters:{$\beta_{1}:0.9$, $\beta_2 :0.999$, $\epsilon : 10^{-8}$ (not very important)}
5. effective. 

In [ ]:
v["dW" + str(l+1)] = beta1*(v["dW" + str(l+1)]) + (1-beta1)*grads["dW" + str(l+1)]
v["db" + str(l+1)] = beta1*(v["db" + str(l+1)]) + (1-beta1)*grads["db" + str(l+1)]

v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / (1 - beta1**t)
v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / (1 - beta1**t)

s["dW" + str(l+1)] = beta2*(s["dW" + str(l+1)]) + (1-beta2)*grads["dW" + str(l+1)]**2
s["db" + str(l+1)] = beta2*(s["db" + str(l+1)]) + (1-beta2)*grads["db" + str(l+1)]**2

s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] / (1 - beta2**t)
s_corrected["db" + str(l+1)] = s["db" + str(l+1)] / (1 - beta2**t)

parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*v_corrected["dW" + str(l+1)]/ (np.sqrt(s_corrected["dW" + str(l+1)]) + epsilon)
parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*v_corrected["db" + str(l+1)]/ (np.sqrt(s_corrected["db" + str(l+1)]) + epsilon)

## 7. Learning Rate Decay
**Background**: As using the mini-batch gradient descent, the algorithm may not converge and wondering around the minima because using the fixed learning rate. 

**Solution**: slowly to reduce the learning rate as doing the training. As approching the minima, the steps taken become a little bit smaller, so the algorithm smaller window. 

$$ \alpha = \frac{1}{1 + \text{decay_rate}*\text{epoch_num}} $$

**Note**:
1. Decay rate is another hyper parameter to tune
2. Other decay methods can also be used. (i.e. Expotentially decay, discrete decay, manual decay, etc)

## 8. Batch Normalization

**Idea**: Since normalizing input featuers will cause the learning more efficient, normalizing the hidden layer may help to increase learning efficiency as well. At same time, also make the weights in the deep layer of the networks more robust to change compare to the first layer of the nerual network.

**Normalization process**:
Given some iteration value in NN $Z^{(1)} \dots Z^{(m)}$:
1. compute mean of Z as $\mu$
2. compute variance of Z as $\sigma^{2}$
3. $Z_{norm}^{(i)} = \frac{Z^{(i)} - \mu}{\sigma^{2}}$
4. $\tilde{Z}^{(i)} = \gamma Z_{norm}^{(i)} + \beta$

**Process**:
```
for t = 1....numOfMiniBatches:
    Compute the forward prop on X^{t}
    In each layer, use batch normalization, replace Z with new Z
    Use backprop to compute dw, d\beta, d\gamma
    Update the parameter
```

**Covariance Shift**: If learned mapping between X and Y, X -> Y, if X's distribution changes, the mapping need to remap. During the nerual network training, the downstream neruons suffers the problem of covariance shift because the coefficients of previous nerouns keep changing. The batch norm help to reduce the distribution of hidden unit values change. 

**Batch Norm as regularization**: Batch norm scaled by mean/variance which cause noises to the values of $Z^{[l]}$, and this has a slight regularization effect similar to the drop out. The regularization of batch norm is relative weak.

**Batch Norm at test**: $\mu = \frac{1}{m}\sum_{i}z^{(i)}$ and variance $\sigma^{2} = \frac{1}{m}\sum_{i}(z^{(i)} - \mu)^2 $is computed over the entire mini-batch, but the test time doesn't have mini-batch. Thus, need to estimate using expontentially weighted average over the layer for all previous mini-batches of the training result. 

**Note**:
1. There is debate on where to apply the normalization, before activation, i.e. $Z^{[l]}$ or after activation $a^{[l]}$. The former utilize more often. 
2. Do not want the mean to always equal to 0 and var =1 since want to take full advantage of the non-linearity of activation function.
3. $\gamma$ and $\beta$ effectively helps to set the mean of $\tilde{Z}$, i.e. when $\gamma = \sqrt{\sigma^2 + \epsilon}$ and $\beta = \mu$, then $\tilde{Z} = Z$
4. The constant $b^{[l]}$ got canceled out because the mean substraction term in batch norm.  => $Z^{[l]} = W^{[l]} => \tilde{Z} = \gamma Z_{norm}^{(i)} + \beta => \beta$ control the bias effect.

## 9. Summary Of Strategies
Many hyper-parameters, some of them are more important to tune than other.

- <span style="color:red"> $\alpha$ </span> (learning rate)
- <span style="color:blue"> $\beta$ </span> (momentum) 
- $\beta_1, \beta_2$ (Adam)
- num of of layers
- <span style="color:blue"> num of hidden units </span>
- learning rate decay
- <span style="color:blue"> mini-batches size </span>

**Random Values Instead of grid**: since hyperparameters have different importance, doesn't make sense say try a single learning rate for different $\epsilon$.

**Coarse to fine**: Once find a good hyperparameter set, zoom into that region and sample more densly

**Sampling random**: for some parameters, sampling uniformly random does not make sense, for example learning rate $\alpha [0.0001, 1]$ , the 90% of the sample will belongs to range (0.1, 1). Try sampling over the log-scale may be a reasonale thing to do.
